In [19]:
#Importing the API and instantiating the REST client according to our keys
from alpaca.trading.client import TradingClient
from alpaca_trade_api.rest import REST, TimeFrame
from websocket import create_connection
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from my_secrets import ALPACA_API_KEY, ALPACA_SECRET_KEY
import json 
import pprint 
import talib 

url = "wss://stream.data.alpaca.markets/v1beta1/crypto"

trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True)

## To Do
Here we would need to know the account position to know if we can make a trade or not.
1. We need to know if we can afford to buy. This means do we have enough cash to make a buy
2. We need to know how much we need to sell of the current position of the asset that we are interested in. How much do we sell the whole position or a percentage.
3. Fix data stream
4. Add headers to data stream
5. 1 and 2 need to be converted to a Python program

Also we would act off of the latest data that we have. If we are running real time by minutes then we need to act within minutes or if we are using end of day closing data that window would be larger.

In [17]:
# BUY ORDER
market_order_buy = MarketOrderRequest(
                      symbol="BTC/USD",
                      qty=1,
                      side=OrderSide.BUY,
                      time_in_force=TimeInForce.GTC
                  )

#SELL ORDER 
market_order_sell = MarketOrderRequest(
                      symbol="BTC/USD",
                      qty=1,
                      side=OrderSide.SELL,
                      time_in_force=TimeInForce.GTC
                  )

In [51]:
api = REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, base_url=url)

# Fetch 1Minute historical bars of Bitcoin
data = api.get_crypto_bars(["BTC/USD"], TimeFrame.Minute).df
# data = data[data.exchange =='CBSE']
data

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-07-29 05:38:00+00:00,29338.200000,29338.200000,29338.200000,1,29338.200000,0.000253,29338.200000,BTC/USD
2023-07-29 05:41:00+00:00,29398.801100,29398.801100,29398.801100,1,29398.801100,0.000161,29398.801100,BTC/USD
2023-07-29 07:08:00+00:00,29373.807651,29373.807651,29373.807651,1,29373.807651,0.000576,29373.807651,BTC/USD
2023-07-29 09:20:00+00:00,29317.599249,29317.599249,29317.599249,1,29317.599249,0.000034,29317.599249,BTC/USD
2023-07-29 09:22:00+00:00,29315.273849,29315.273849,29315.273849,1,29315.273849,0.000068,29315.273849,BTC/USD
2023-07-29 09:39:00+00:00,29308.638827,29308.638827,29308.638827,1,29308.638827,0.000170,29308.638827,BTC/USD
2023-07-29 09:46:00+00:00,29309.041405,29309.041405,29309.041405,1,29309.041405,0.000134,29309.041405,BTC/USD
2023-07-29 10:20:00+00:00,29294.300000,29294.300000,29294.300000,1,29294.300000,0.001613,29294.300000,BTC/USD
2023-07-29 11:48:00+00:00,29303.289000,29303.289000,29303.289000,1,29303.289000,0.016355,29303.289000,BTC/USD


In [52]:
#DEFINE THE MACD AND SELL/BUY SIGNALS
exp1 = data['close'].ewm(span=12).mean()
exp2 = data['close'].ewm(span=26).mean() 
macd = exp1 - exp2
signal = macd.ewm(span=9).mean()


msell = signal >= 6
mbuy = signal <= -4
mbuy


timestamp
2023-07-29 05:38:00+00:00    False
2023-07-29 05:41:00+00:00    False
2023-07-29 07:08:00+00:00    False
2023-07-29 09:20:00+00:00    False
2023-07-29 09:22:00+00:00    False
2023-07-29 09:39:00+00:00    False
2023-07-29 09:46:00+00:00    False
2023-07-29 10:20:00+00:00    False
2023-07-29 11:48:00+00:00    False
2023-07-29 12:00:00+00:00    False
2023-07-29 12:10:00+00:00     True
2023-07-29 12:11:00+00:00     True
2023-07-29 12:24:00+00:00     True
2023-07-29 12:43:00+00:00     True
2023-07-29 13:00:00+00:00     True
2023-07-29 13:34:00+00:00     True
2023-07-29 13:36:00+00:00     True
2023-07-29 13:49:00+00:00    False
2023-07-29 14:06:00+00:00    False
2023-07-29 14:22:00+00:00    False
2023-07-29 14:54:00+00:00    False
2023-07-29 15:04:00+00:00    False
2023-07-29 15:56:00+00:00    False
2023-07-29 16:02:00+00:00    False
2023-07-29 16:06:00+00:00    False
Name: close, dtype: bool

In [53]:
latestBuySignal = mbuy[22]

In [54]:
#buying specified security
# api = REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, base_url=url)

# Fetch 1Minute historical bars of Bitcoin
# data = api.get_crypto_bars(["BTC/USD"], TimeFrame.Minute).df
# data = data[data.exchange =='CBSE']



# def MACD(self):
if msell is True:
    market_order = trading_client.submit_order(market_order_sell)
    for property_name, value in market_order:
        print(f"\"{property_name}\": {value}")
elif latestBuySignal is True:
    market_order = trading_client.submit_order(market_order_buy)
    for property_name, value in market_order:
        print(f"\"{property_name}\": {value}")
else:
    print("Not buying BTC/USD")

# MACD()


# positions = trading_client.get_all_positions()
# # for position in positions:
# #     for property_name, value in position:
# #         print(f"\"{property_name}\": {value}")
#
# positions

"id": 80ab9c88-f210-490e-a1d7-9230108d7ed2
"client_order_id": e817d39b-087c-496a-90ee-53554849a472
"created_at": 2023-07-29 16:42:12.181365+00:00
"updated_at": 2023-07-29 16:42:12.181412+00:00
"submitted_at": 2023-07-29 16:42:12.180284+00:00
"filled_at": None
"expired_at": None
"canceled_at": None
"failed_at": None
"replaced_at": None
"replaced_by": None
"replaces": None
"asset_id": 276e2673-764b-4ab6-a611-caf665ca6340
"symbol": BTC/USD
"asset_class": crypto
"notional": None
"qty": 1
"filled_qty": 0
"filled_avg_price": None
"order_class": simple
"order_type": market
"type": market
"side": buy
"time_in_force": gtc
"limit_price": None
"stop_price": None
"status": pending_new
"extended_hours": False
"legs": None
"trail_percent": None
"trail_price": None
"hwm": None


In [34]:
links = https://analyzingalpha.com/python-trading-tools

SyntaxError: invalid syntax (48376082.py, line 1)

In [150]:
positions = trading_client.get_all_positions()
for position in positions:
    for property_name, value in position:
        print(f"\"{property_name}\": {value}")

In [79]:
from alpaca.trading.requests import GetAssetsRequest

# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change}')

Today's portfolio balance change: $-187.9278273186792
